In [3]:
### Run this cell before continuing.
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source('tests.R')
source('cleanup.R')

Warning message in file(filename, "r", encoding = encoding):
“cannot open file 'tests.R': No such file or directory”


ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [7]:
# Reading dataset into R
sleep <- read_csv("https://raw.githubusercontent.com/mikkielynn/DSCI_100_project/main/Sleep_health_and_lifestyle_dataset.csv")
head(sleep)

Rows: 374 Columns: 13
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (5): Gender, Occupation, BMI Category, Blood Pressure, Sleep Disorder
dbl (8): Person ID, Age, Sleep Duration, Quality of Sleep, Physical Activity...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>
1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,None
2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,None
3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,None
4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
6,Male,28,Software Engineer,5.9,4,30,8,Obese,140/90,85,3000,Insomnia


In [8]:
# Renaming variable names
colnames(sleep) <- c("Person_ID", "Gender", "Age", "Occupation", "Sleep_Duration", "Quality_of_sleep", "Physical_Activity_Level", "Stress_Level", "BMI", "Blood_Pressure", "Heart_Rate", "Daily_steps", "Sleep_disorder")
sleep

Person_ID,Gender,Age,Occupation,Sleep_Duration,Quality_of_sleep,Physical_Activity_Level,Stress_Level,BMI,Blood_Pressure,Heart_Rate,Daily_steps,Sleep_disorder
<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>
1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,None
2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,None
3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,None
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
372,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
373,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
374,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea


In [9]:
# Tidied data; selected columns used for analysis
sleep_2 <- sleep |>
    select(-Person_ID, -Gender, -Occupation, -BMI, -Blood_Pressure)
sleep_2

Age,Sleep_Duration,Quality_of_sleep,Physical_Activity_Level,Stress_Level,Heart_Rate,Daily_steps,Sleep_disorder
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
27,6.1,6,42,6,77,4200,None
28,6.2,6,60,8,75,10000,None
28,6.2,6,60,8,75,10000,None
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
59,8.1,9,75,3,68,7000,Sleep Apnea
59,8.1,9,75,3,68,7000,Sleep Apnea
59,8.1,9,75,3,68,7000,Sleep Apnea


In [10]:
#splitting code
sleep_split <- initial_split(sleep_2, prop = 0.5, strata = Sleep_disorder)
sleep_train <- training(sleep_split)
sleep_test <- testing(sleep_split)
sleep_train     
sleep_test

#ensuring we have equal proportions of the different lables for the sleep disorder
sleep_train |>
    count(Sleep_disorder)

sleep_test |>
    count(Sleep_disorder)

Age,Sleep_Duration,Quality_of_sleep,Physical_Activity_Level,Stress_Level,Heart_Rate,Daily_steps,Sleep_disorder
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
29,6.3,6,40,7,82,3500,Insomnia
30,6.4,5,35,7,78,4100,Insomnia
33,6.0,6,30,8,72,5000,Insomnia
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
59,8.0,9,75,3,68,7000,Sleep Apnea
59,8.1,9,75,3,68,7000,Sleep Apnea
59,8.1,9,75,3,68,7000,Sleep Apnea


Age,Sleep_Duration,Quality_of_sleep,Physical_Activity_Level,Stress_Level,Heart_Rate,Daily_steps,Sleep_disorder
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
28,6.2,6,60,8,75,10000,None
28,5.9,4,30,8,85,3000,Sleep Apnea
28,5.9,4,30,8,85,3000,Sleep Apnea
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
59,8.1,9,75,3,68,7000,Sleep Apnea
59,8.0,9,75,3,68,7000,Sleep Apnea
59,8.1,9,75,3,68,7000,Sleep Apnea


Sleep_disorder,n
<chr>,<int>
Insomnia,38
None,109
Sleep Apnea,39


Sleep_disorder,n
<chr>,<int>
Insomnia,39
None,110
Sleep Apnea,39


In [10]:
# Summarizing means across quantitative variables
sleep_summary <- sleep_2 |>
    summarize(across(Age:Daily_steps, mean))
sleep_summary

Age,Sleep_Duration,Quality_of_sleep,Physical_Activity_Level,Stress_Level,Heart_Rate,Daily_steps
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
42.18449,7.132086,7.312834,59.17112,5.385027,70.16578,6816.845
